In [ ]:
import pandas as pd
modeling_df = pd.read_csv('modeling_df.csv',index_col=0)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(modeling_df.iloc[:,[0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56]],modeling_df['result'],test_size=0.25,stratify=modeling_df['result'], random_state=123)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.3, stratify = y_test, random_state=123)

In [ ]:
# Importing dataset
from pycaret.datasets import get_data
diabetes = get_data('diabetes')

# Importing module and initializing setup
from pycaret.classification import *
clf1 = setup(data = diabetes, target = 'Class variable')


ModuleNotFoundError: No module named 'pycaret'

In [ ]:
#knn
import optuna, matplotlib.pyplot as plt
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.neighbors import KNeighborsClassifier
#1 k-fold
kfold = KFold(n_splits=5,random_state=123, shuffle=True)
#2 knn
def knn_objective(trial):
    params= {'algorithm' :  trial.suggest_categorical('algorithm', ['auto','ball_tree','kd_tree','brute']),
             'n_neighbors' : trial.suggest_int('n_neighbors', 3, 13, step=2)
            }
    knn = KNeighborsClassifier(n_jobs=-1, **params)
    score = cross_val_score(knn, X_train, y_train, cv=kfold, scoring="accuracy")
    accuracy = score.mean()
    return accuracy
knn_study = optuna.create_study(direction="maximize")
knn_study.optimize(knn_objective, n_trials=50)
knn_study.optimize(knn_objective, n_trials=10)
knn_study.best_trial.params
knn_study.best_trial.values

print(optuna.visualization.plot_param_importances(knn_study))
opt_rf = KNeighborsClassifier(n_jobs=-1, random_state=123,
                                criterion = knn_study.best_params["algorithm"],
                                n_neighbors = knn_study.best_params["n_neighbors"] #fix overfitting
                                )
opt_rf.fit(X_train, y_train)
print(accuracy_score(y_test, opt_rf.predict(X_test)))

In [ ]:
#RandomForest
import optuna, matplotlib.pyplot as plt
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
#1 k-fold
kfold = KFold(n_splits=5, random_state=123, shuffle=True)
#2 RandomForest
def rf_objective(trial):
    params = {
        "criterion" : trial.suggest_categorical("criterion", ["gini","entropy"]),
        "n_estimators" : trial.suggest_int("n_estimators", 100,500),
        "max_depth" : trial.suggest_int("max_depth", 1, 100),
        "max_leaf_nodes":trial.suggest_int("max_leaf_nodes", 2, 1000),
        "max_features" : trial.suggest_int("max_features", 1, X_train.shape[1]),
        "min_samples_leaf":trial.suggest_int("min_samples_leaf", 1, 50)
    }
    rf = RandomForestClassifier(n_jobs=-1, random_state=123, **params)
    score = cross_val_score(rf, X_train, y_train, cv=kfold, scoring="accuracy")
    accuracy = score.mean()
    return accuracy
rf_study = optuna.create_study(direction="maximize")
print(rf_study.optimize(rf_objective, n_trials=10))
print(rf_study.best_trial.params)
print(rf_study.best_trial.values)
print(optuna.visualization.plot_param_importances(rf_study))
#3 optimization
opt_rf = RandomForestClassifier(n_jobs=-1,
                                random_state=123,
                                criterion = rf_study.best_params["criterion"],
                                n_estimators = rf_study.best_params["n_estimators"],
                                max_depth = rf_study.best_params["max_depth"],
                                max_leaf_nodes = rf_study.best_params["max_leaf_nodes"],
                                max_features = rf_study.best_params["max_features"],
                                min_samples_leaf = rf_study.best_params["min_samples_leaf"]
                                )
opt_rf.fit(X_train, y_train)
print(accuracy_score(y_test, opt_rf.predict(X_test)))

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
#xgboost
import optuna, matplotlib.pyplot as plt
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score
from xgboost import XGBClassifier
#1 k-fold
kfold = KFold(n_splits=5,random_state=123, shuffle=True)
#2 xgboost
def xgb_objective(trial):
    params = {
        "booster" : trial.suggest_categorical("booster", ["gbtree","gblinear","dart"]),
        "n_estimators" : trial.suggest_int("n_estimators", 100,500),
        "learning_rate" : trial.suggest_loguniform("learning_rate", 1e-3, 8e-1),
        "max_depth" : trial.suggest_int("max_depth", 1, 100),
        "subsample" : trial.suggest_discrete_uniform("subsample", 0.1, 1, 0.05),
        "colsample_bytree":trial.suggest_discrete_uniform("colsample_bytree", 0.1,1, 0.05),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.5, 1.0),
        "colsample_bynode": trial.suggest_float("colsample_bynode", 0.5, 1.0),
        "reg_alpha" : trial.suggest_loguniform("reg_alpha", 1e-2, 1e1),
        "reg_lambda" : trial.suggest_loguniform("reg_lambda", 1e-2, 1e1),
        'min_child_weight': trial.suggest_int('min_child_weight', 2, 15),
        "gamma": trial.suggest_float("gamma", 0.1, 1.0, log=True)
    }
    xgbc = XGBClassifier(n_jobs = 2,random_state=123, use_label_encoder=False, **params,
        objective = "multi:softmax", eval_metric  ="error") # multi:softmax이거 일듯
    score= cross_val_score(xgbc, X_train, y_train, cv=kfold, scoring="accuracy")
    accuracy = score.mean()
    return accuracy
xgbc_study = optuna.create_study(direction="maximize")
print(xgbc_study.optimize(xgb_objective, n_trials=10))
print(xgbc_study.best_trial.params)
print(xgbc_study.best_trial.values)
#3 optimization
optuna.visualization.plot_param_importances(xgbc_study)
opt_xgbc = XGBClassifier( 
                        n_jobs= 2,
                        random_state=123,
                        booster=xgbc_study.best_params["booster"],
                        n_estimators = xgbc_study.best_params["n_estimators"],
                        learning_rate = xgbc_study.best_params["learning_rate"],
                        max_depth = xgbc_study.best_params["max_depth"],
                        subsample = xgbc_study.best_params["subsample"],
                        colsample_bytree = xgbc_study.best_params["colsample_bytree"],
                        reg_alpha = xgbc_study.best_params["reg_alpha"],
                        reg_lambda = xgbc_study.best_params["reg_lambda"],
                        use_label_encoder=False,
                        objective = "multi:softmax", # multi:softmax이거 일듯
                        eval_metric = "error")
opt_xgbc.fit(X_train, y_train)
print(accuracy_score(y_test, opt_xgbc.predict(X_test)))

[I 2022-11-16 17:06:42,265] A new study created in memory with name: no-name-13eb0c28-4701-4a97-b12c-75bdc4f78da9
C:\Users\dlwjd\AppData\Local\Temp\ipykernel_1076\1500963839.py:16: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  "learning_rate" : trial.suggest_loguniform("learning_rate", 1e-3, 8e-1),
C:\Users\dlwjd\AppData\Local\Temp\ipykernel_1076\1500963839.py:18: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  "subsample" : trial.suggest_discrete_uniform("subsample", 0.1, 1, 0.05),
C:\Users\dlwjd\AppData\Local\Temp\ipykernel_1076\1500963839.py:19: FutureWarning: suggest_discrete_uniform has been deprecated in v3.0.0. This feature will be r

In [ ]:
#AdaBoost
import optuna, matplotlib.pyplot as plt
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.ensemble import AdaBoostClassifier
#1 k-fold
kfold = KFold(n_splits=5, random_state=123, shuffle=True)
#2 adaboost
def ada_objective(trial):
    params = { #알고리즘은 디폴트값이 다른값들 보다 좋음, 다른거는 이항연산자임 파라미터가 없음 그래서 굳이 필없
        "n_estimators" : trial.suggest_int("n_estimators", 50,500),
        "learning_rate" : trial.suggest_loguniform("learning_rate", 1e-3, 8e-1)
    }
    ada = AdaBoostClassifier(n_jobs = -1, random_state=123,**params)
    score = cross_val_score(ada, X_train, y_train, cv=kfold, scoring="accuracy")
    accuracy = score.mean()
    return accuracy
ada_study = optuna.create_study(direction="maximize")
print(ada_study.optimize(ada_objective, n_trials=10))
print(ada_study.best_trial.params)
print(ada_study.best_trial.values)
#3 optimization
optuna.visualization.plot_param_importances(ada_study)
opt_ada = AdaBoostClassifier( 
                        n_jobs= -1,
                        random_state=123,
                        n_estimators = ada_study.best_params["n_estimators"],
                        learning_rate = ada_study.best_params["learning_rate"]
                        )
opt_ada.fit(X_train, y_train)
print(accuracy_score(y_test, opt_ada.predict(X_test)))

In [ ]:
#SVM
import optuna, matplotlib.pyplot as plt
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.svm import SvcClassifier
#1 k-fold
kfold = KFold(n_splits=5, random_state=123, shuffle=True)
#SVC
def svc_objective(trial):
    params = {
        'kernel': trial.suggest_categorical('kernel', ['linear','rbf','sigmoid']),
        'C': trial.suggest_loguniform('C', 1e+0, 1e+2/2),
        'degree' :trial.suggest_int("degree", 0,6),
        'gamma': trial.suggest_loguniform('gamma', 1e-3, 3.0)
    }
    svc = SvcClassifier(n_jobs = -1, random_state=123,**params)
    score = cross_val_score(svc, X_train, y_train, cv=kfold, scoring="accuracy")
    accuracy = score.mean()
    return accuracy
svc_study = optuna.create_study(direction="maximize")
print(svc_study.optimize(svc_objective, n_trials=10))
print(svc_study.best_trial.params)
print(svc_study.best_trial.values)
#3 optimization
optuna.visualization.plot_param_importances(svc_study)
opt_svc = AdaBoostClassifier( 
                        n_jobs= -1,
                        random_state=123,
                        kernel = svc_study.best_params["kernel"],
                        C = svc_study.best_params["C"],
                        degree = svc_study.best_params["degree"],
                        gamma = svc_study.best_params["gamma"]
                        )
opt_svc.fit(X_train, y_train)
print(accuracy_score(y_test, opt_svc.predict(X_test)))

In [ ]:
# 분류
# K-최근접 이웃 : K-Nearest Neighbors
# 선형 회귀 : Linear Regression
# 로지스틱 회귀 : Logistic Regression
# 결정 트리 : Decision Tree
# 랜덤 포레스트 : Random Forest
# 점진적 부스팅 머신: GBM(Gradient boosting Machine)
# 서포트 벡터 머신 : SVM(Support Vector Machine)
# 신경망 : Neural Network